In [ ]:
# 📦 Imports – alle gesammelt oben
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 📂 Daten laden
df = pd.read_csv('../data/raw/creditcard.csv')

# 🔍 Grundstruktur prüfen
print("Shape:", df.shape)
display(df.head())


In [ ]:
# Klassenverteilung zählen
print(df['Class'].value_counts())

# Prozentual
fraud_pct = df['Class'].value_counts(normalize=True) * 100
print("Prozentuale Verteilung:\n", fraud_pct)

# Visualisierung
plt.figure(figsize=(6,4))
sns.countplot(data=df, x='Class')
plt.title('Verteilung: Legit vs Fraud')
plt.xticks([0, 1], ['Legit (0)', 'Fraud (1)'])
plt.show()



In [ ]:
# 'Time' ist in Sekunden seit Start → konvertieren in Stunden (0–23)
df['Hour'] = ((df['Time'] // 3600) % 24).astype(int)

# Legit vs Fraud pro Stunde
plt.figure(figsize=(10,5))
sns.histplot(data=df[df['Class'] == 0], x='Hour', bins=24, color='blue', label='Legit', stat='density', kde=True)
sns.histplot(data=df[df['Class'] == 1], x='Hour', bins=24, color='red', label='Fraud', stat='density', kde=True)
plt.title('Verteilung der Transaktionen über den Tag')
plt.xlabel('Stunde des Tages')
plt.legend()
plt.show()



In [ ]:
tx_per_hour = df.groupby('Hour').size()

plt.figure(figsize=(10,4))
tx_per_hour.plot(kind='bar', color='gray')
plt.title('Anzahl der Transaktionen je Stunde')
plt.xlabel('Stunde des Tages')
plt.ylabel('Anzahl')
plt.show()


In [ ]:
# Korrelationen mit Zielspalte anzeigen
correlation = df.corr(numeric_only=True)['Class'].sort_values(ascending=False)
print("Top positive Korrelationen:\n", correlation.head(10))
print("\nTop negative Korrelationen:\n", correlation.tail(10))

# Heatmap
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(numeric_only=True), cmap='coolwarm_r', annot=False)
plt.title('Korrelationen der Features')
plt.show()


In [ ]:
# Frisches DataFrame zur Sicherheit
df_pca = df.copy()

# Ziel- und Nicht-Zielspalten trennen
X = df_pca.drop(columns=['Class', 'Time', 'Hour'])
y = df_pca['Class']

# Skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA auf 2 Komponenten
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# DataFrame für Plot
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
pca_df['Class'] = y.values

# Plot
plt.figure(figsize=(8,6))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='Class', alpha=0.6, palette='coolwarm')
plt.title('PCA der Transaktionen')
plt.show()

# Erklärte Varianz anzeigen
explained = pca.explained_variance_ratio_
print(f"Erklärte Varianz (PC1 & PC2): {explained[0]:.4f}, {explained[1]:.4f}  → Gesamt: {explained.sum():.4f}")
